In [57]:
import os, json, copy
os.chdir("C:\\Users\\Nathan\\Documents\\Dropbox\\PM\\CUT MY DOC INTO PIECES2")

In [38]:
def getj():
    with open("PMForms2.json") as f:
        return json.load(f, object_pairs_hook=Section)

In [103]:
class Spec2():
    def __init__(self, obj):
        self.name = obj['Name']
        self.template = obj.get('Template')
        self.document_number = obj('DocumentNumber')
        self.properties = Properties(obj.get('Properties'))
        self.parent_spec = obj.get('ParentSpec', [])
        self.is_mixin = obj.get('IsMixin', False)
        

class Spec():
    def __init__(self, obj):
        self._obj = obj
        self._parents = []
        
    def __getitem__(self, key):
        return self.lookup(key)
    
    def get(self, key, default=None):
        try:
            return self.lookup(key)
        except KeyError:
            return default
    
    def lookup(self, key):
        path = key.split(".")
        try:
            return self._lookup(path)
        except KeyError:
            raise KeyError(key) from None

    def _lookup(self, path):
        return self._internal_lookup(path)
    
    def _internal_lookup(self, path):
        v = self._obj
        for p in path:
            v = v[p]
        return v
    
    def __repr__(self):
        return "<Spec: %s>"%self['Name']
    
    def keys(self):
        return self._obj.keys()
    
    def values(self):
        return self._obj.values()
    
    def items(self):
        return self._obj.items()
    
    def __iter__(self):
        return iter(self._obj)
            
def merge(src, dst):
    for k,v in src.items():
        if isinstance(v, dict):
            if k not in dst:
                dst[k] = {}
            merge(v, dst[k])
        else:
            dst[k] = v
                

def specmerge(allspecs, lro, spec, parent):
    sspec = copy.deepcopy(spec._obj)
    merge(parent._obj, sspec)
    pparents = lro[parent['Name']]
    for pp in pparents:
        pparent = allspecs[pp]
        specmerge(allspecs, lro, spec, pparent)
    
                
def specmergeall(allspecs, name, parents, lro):
    spec = allspecs[name]
    for p in reversed(parents):
        parent = allspecs[p]
        specmerge(allspecs, lro, spec, parent)
    
                
def loadj():
    j = getj()
    rawspecs = j['Specs']
    rawtargets = j['Targets']
    allspecs = {}
    targets = []
    for spec in rawspecs:
        s = Spec(spec)
        allspecs[s['Name']] = s
    for target in rawtargets:
        t = Spec(target)
        allspecs[t['Name']] = t
        targets.append(t)
    
    _lro_ = {}
    for s in allspecs.values():
        pspecs = s.get('ParentSpec', [])
        _lro_[s['Name']] = []
        if not isinstance(pspecs, list):
            print("??? dum udm", s['Name'])
        for ps in pspecs:
            _lro_[s['Name']].append(ps)
            
    for k,v in _lro_.items():
        specmergeall(allspecs, k, v, _lro_)
    
    return targets  # mutated in-place 
            
def loadj3():
    j = getj()
    rawspecs = j['Specs']
    rawtargets = j['Targets']
    allspecs = {}
    targets = []
    for spec in rawspecs:
        s = Spec(spec)
        allspecs[s['Name']] = s
    for target in rawtargets:
        t = Spec(target)
        allspecs[t['Name']] = t
        targets.append(t)
    
    _lro_ = {}
    for s in allspecs.values():
        _lro_[s['Name']] = s.get('ParentSpec', [])
            
    for k,v in _lro_.items():
        specmergeall(allspecs, k, v, _lro_)
    
    return targets  # mutated in-place 

    
def iter_parents(allspecs, specname):
    seen = set()
    spec = allspecs[specname]
    
    
    
class SanityError(Exception):
    pass
    
def loadj2():
    j = getj()
    rawspecs = j['Specs']
    rawtargets = j['Targets']
    allspecs = {}
    targets = []
    
    for spec in rawspecs:
        s = Spec(spec)
        allspecs[s['Name']] = s
    
    for target in rawtargets:
        t = Spec(target)
        allspecs[t['Name']] = t
        targets.append(t)
    
    for s in allspecs.values():
        for p in s.get('ParentSpec', []):
            s._parents.append(allspecs[p])
            
    # sanity checks
    for t in targets:
        sections = set(t['Properties.Sections'])
        for tr in t.get('Properties.TextReplace', {}).keys():
            if tr not in sections and not t.get('IsMixin', False):
                raise SanityError("TextReplace for %r contains unknown section: %r"%(t['Name'], tr))
        
    return targets

In [116]:
def merge(src, dst):
    for k,v in src.items():
        if isinstance(v, dict):
            if k not in dst:
                dst[k] = {}
            merge(v, dst[k])
        else:
            dst[k] = v

def merge_parents(props, parent, seen, specs):
    for p in reversed(parent.get("ParentSpec", [])):
        if p in seen: continue
        np = specs[p]
        merge_parents(props, np, seen, specs)
    merge(parent, props)
    print("merged", parent['Name'])
    seen.add(parent['Name'])

def loadj4():
    j = getj()
    rawspecs = j['Specs']
    rawtargets = j['Targets']
    allspecs = {}
    targets = []
    
    for spec in rawspecs:
        s = Spec(spec)
        allspecs[s['Name']] = s
    
    for target in rawtargets:
        t = Spec(target)
        allspecs[t['Name']] = t
        targets.append(t)
    
    for target in targets:
        new = {}
        seen = set()
        merge_parents(new, target._obj, seen, allspecs)
        target._obj = new
        print()
        
    return targets

In [117]:
loadj4()

merged PBS 3 MAG
merged IA-3-B-5xx

merged PBS 3 MAG
merged Hamilton SU
merged IA-3-B-7xx

merged PBS 15 MAG
merged IA-15-B-5xx

merged PBS 15 MAG
merged Hamilton SU
merged IA-15-B-7xx

merged PBS 15 MAG
merged PBS 80 MAG
merged IA-80-B-5xx

merged PBS 15 MAG
merged PBS 80 MAG
merged Hamilton SU
merged IA-80-B-7xx



[<Spec: IA-3-B-5xx>,
 <Spec: IA-3-B-7xx>,
 <Spec: IA-15-B-5xx>,
 <Spec: IA-15-B-7xx>,
 <Spec: IA-80-B-5xx>,
 <Spec: IA-80-B-7xx>]

In [114]:
specs = loadj4()

merged PBS 3 MAG
merged IA-3-B-5xx

merged PBS 3 MAG
merged Hamilton SU
merged IA-3-B-7xx

merged PBS 15 MAG
merged IA-15-B-5xx

merged PBS 15 MAG
merged Hamilton SU
merged IA-15-B-7xx

merged PBS 15 MAG
merged PBS 80 MAG
merged IA-80-B-5xx

merged PBS 15 MAG
merged PBS 80 MAG
merged Hamilton SU
merged IA-80-B-7xx



In [115]:
specs[-1]['Properties.TextReplace']

{'Heater Resistance': {'{FHAccRange}': '(110v: 155-165) (220v: 755-765)',
  '{MHAccRange}': '(110v: 13.68-15.84) (220v: 54.72-63.36)'},
 'Level Calibration_15L': {'{IMProc}': 'IM00164',
  '{v1}': '20',
  '{v2}': '40',
  '{v3}': '60'},
 'Level Verification': {'{volume}': '50'},
 'MFC Accuracy': {'{AirAcc1}': '1.90 - 2.10',
  '{AirAcc2}': '4.80 - 5.20',
  '{AirSP1}': '2.0',
  '{AirSP2}': '5.0',
  '{CO2Acc1}': '0.475 - 0.525',
  '{CO2Acc2}': '0.950 - 1.050',
  '{CO2SP1}': '0.50',
  '{CO2SP2}': '1.0',
  '{N2Acc1}': '1.90 - 2.10',
  '{N2Acc2}': '4.80 - 5.20',
  '{N2SP1}': '2.0',
  '{N2SP2}': '5.0',
  '{O2Acc1}': '1.90 - 2.10',
  '{O2Acc2}': '4.80 - 5.20',
  '{O2SP1}': '2.0',
  '{O2SP2}': '5.0',
  '{unit}': 'LPM'},
 'MFC Calibration': {'{Air1}': '1.0',
  '{Air2}': '2.0',
  '{Air3}': '3.0',
  '{Air4}': '5.0',
  '{Air5}': '10.0',
  '{CO21}': '0.50',
  '{CO22}': '0.75',
  '{CO23}': '1.0',
  '{CO24}': '1.5',
  '{CO25}': '2.0',
  '{N21}': '1.0',
  '{N22}': '2.0',
  '{N23}': '3.0',
  '{N24}': '5.0